In [10]:
import scvi
import scanpy as sc
import anndata
import skmisc
import numpy as np

In [11]:
adata = sc.read_h5ad('/home/ec2-user/scrnaseq_tcsl154/rds/scrna.wnn.all.h5ad')

In [12]:
filter_result = sc.pp.filter_genes(adata, min_counts=3, inplace=False)
sc.pp.filter_genes(adata, min_counts=3)

/home/ec2-user/anaconda3/envs/scvi-env/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


In [13]:
adata.layers["counts"] = adata.raw[:, filter_result[0]].X.copy() # preserve counts
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [14]:
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=2000,
    subset=True,
    layer="counts",
    flavor="seurat_v3"
)

/home/ec2-user/anaconda3/envs/scvi-env/lib/python3.7/site-packages/scanpy/preprocessing/_highly_variable_genes.py:144: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  df.loc[: int(n_top_genes), 'highly_variable'] = True


In [15]:
scvi.data.setup_anndata(adata, labels_key='sample_num', layer="counts")
# create the model
model = scvi.model.SCVI(adata)

INFO      No batch_key inputted, assuming all cells are same batch                           
INFO      Using labels from adata.obs["sample_num"]                                          
INFO      Using data from adata.layers["counts"]                                             
INFO      Computing library size prior per batch                                             


/home/ec2-user/anaconda3/envs/scvi-env/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


INFO      Successfully registered anndata object containing 41937 cells, 2000 genes, 1       
          batches, 16 labels, and 0 proteins. Also registered 0 extra categorical covariates 
          and 0 extra continuous covariates.                                                 
INFO      Please do not further modify adata until model is trained.                         


In [ ]:
model.train()

INFO      KL warmup for 400 epochs                                                           


Output()

In [20]:
model.save("scvi_model/")

In [21]:
model = scvi.model.SCVI.load(adata=adata, dir_path="scvi_model/", use_cuda=True)

In [22]:
model

SCVI Model with params: 
n_hidden: 128, n_latent: 10, n_layers: 1, dropout_rate: 0.1, dispersion: gene, gene_likelihood: zinb, latent_distribution: normal
Training status: Trained

In [23]:
latent = model.get_latent_representation()
adata.obsm["X_scVI"] = latent

In [27]:
import pandas as pd

In [40]:
pd.DataFrame(latent.transpose()).to_csv('scrna_latent_space.csv.gz', index=False, compression='gzip', header=False)

In [30]:
pd.DataFrame(adata.layers["counts"]).to_csv('scrna_counts.csv')

In [25]:
adata.layers["counts"].shape

(41937, 2000)